In [1]:
import numpy
from collections import deque
import pandas
import math
import pandas_datareader.data as web
import datetime
import requests
import requests_cache
import xlrd
import tempfile
import itertools

/Users/justus/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
def get_japan_rates():
    url = 'http://www.mof.go.jp/english/jgbs/reference/interest_rate/historical/jgbcme_all.csv'
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)

    r = session.get(url, stream=True)

    with tempfile.NamedTemporaryFile(suffix='.csv') as tmp:
        for chunk in r.iter_content(chunk_size=1024):
            tmp.write(chunk)
            
        tmp.flush()
        
        df = pandas.read_csv(tmp.name, header=1, index_col=0, parse_dates=True, na_values='-')


    # The MOF data is daily. Which is overkill. We just want to rate at the
    # start of each month.
    df = df.groupby(by=lambda x: datetime.date(x.year, x.month, 1)).first()

    # The MOF data reports rates at 7.98 instead of 0.0798, which is what we want
    df = df.applymap(lambda x: x/100)
    return df

all_by_month = get_japan_rates()
all_by_month.head()

,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,15Y,20Y,25Y,30Y,40Y
1974-09-01,0.10327,0.09362,0.08830,0.08515,0.08348,0.08290,0.08240,0.08121,0.08127,NaN,NaN,NaN,NaN,NaN,NaN
1974-10-01,0.10374,0.09374,0.08837,0.08519,0.08354,0.08298,0.08244,0.08119,0.08201,NaN,NaN,NaN,NaN,NaN,NaN
1974-11-01,0.10596,0.09426,0.08868,0.08538,0.08367,0.08305,0.08245,0.08127,0.08226,NaN,NaN,NaN,NaN,NaN,NaN
1974-12-01,0.10714,0.09527,0.08891,0.08566,0.08397,0.08325,0.08249,0.08120,0.08238,NaN,NaN,NaN,NaN,NaN,NaN
1975-01-01,0.10941,0.09572,0.08922,0.08580,0.08402,0.08326,0.08250,0.08133,0.08227,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def iterate_fund(ladder, yield_curve, max_maturity):
    ladder.reduce_maturities()
    ladder.generate_payments()
#    print(ladder.cash)
#    import pdb;pdb.set_trace()
    sold_bonds = ladder.sell_bonds(yield_curve)

    # Only buy a new bond if we actually sold one...
    if sold_bonds:
        ladder.buy_bond(yield_curve[max_maturity-1], max_maturity)
    
    # This happens *after* we sell the shortest bond and buy a new long one
    # (at least, that's what longinvest %doctest_modees...)
    nav = ladder.get_nav(yield_curve)

    return (ladder, nav)

In [4]:
def a2m(annual_rate):
    return pow(annual_rate + 1, 1/12) - 1

class Bond:
    def __init__(self, face_value, yield_pct, maturity, payments_per_year=12):
        self.face_value = face_value
        self.yield_pct = yield_pct
        self.maturity = maturity
        self.payments_per_year = payments_per_year
        
    def __repr__(self):
        return ('Maturity: %d | Yield: %.2f%% | Face Value: $%.2f' % (self.maturity, self.yield_pct * 100, self.face_value))

    def gen_payment(self):
        return self.face_value * self.yield_pct / self.payments_per_year
    
    def value(self, rates):
        value = numpy.pv(rates[self.maturity - 1], self.maturity / 12, (self.face_value * self.yield_pct), self.face_value)
        return -value
    
class BondLadder:
    def __init__(self, min_maturity, max_maturity):
        self.min_maturity = min_maturity
        self.max_maturity = max_maturity
        self.cash = 0
        
        self.ladder = set()
        
    def print_all(self):
        for bond in sorted(self.ladder, key=lambda b: b.maturity):
            print(bond)
            
    def print_all_values(self, rates):
        for bond in sorted(self.ladder, key=lambda b: b.maturity):
            print(bond.value(rates))
        
    def buy_bond(self, rate, maturity):
        b = Bond(self.cash, rate, maturity)
        self.add_bond(b)
        self.cash = 0
        return b
        
    def get_nav(self, rates):
        return self.cash + sum((b.value(rates) for b in self.ladder))

    def generate_payments(self):
        self.cash += sum((b.gen_payment() for b in self.ladder))        
        
    def __repr__(self):
        return ('%d-%d Ladder { Num Bonds: %d. }' % (self.max_maturity, self.min_maturity, len(self.ladder)))
        
    def add_bond(self, bond):
        #assert bond.maturity <= self.max_maturity
        #assert bond.maturity >= self.min_maturity
        self.ladder.add(bond)
    
    def reduce_maturities(self):
        for bond in self.ladder:
            bond.maturity -= 1

    def sell_bonds(self, rates):
        to_sell = filter(lambda bond: bond.maturity <= self.min_maturity, self.ladder)
        to_sell = list(to_sell)
        self.ladder = self.ladder.difference(to_sell)
        self.cash += sum((b.value(rates) for b in to_sell))
        return to_sell

In [5]:
def bootstrap(yield_curve, max_bonds, min_maturity):
    bond_yield = yield_curve[max_bonds - 1]

    # Why - 11?
    #min_maturity -= 11

    ladder = BondLadder(min_maturity, max_bonds)
    starting_face_value = 50 # chosen arbitrarily (to match longinvest)

    for i, j in zip(range(max_bonds), range(min_maturity, max_bonds+1)):
        face_value = pow(1 + a2m(bond_yield), i) * starting_face_value
        b = Bond(face_value, bond_yield, j)
        ladder.add_bond(b)
    return ladder
bootstrap([.0532]*120, 10*12, 5*12)

120-60 Ladder { Num Bonds: 61. }

In [6]:
def splice_data(raw_rates, series):
    # Start by loading the data we get from Shiller.
    # This will always exist.

    def safe_add(series_index, rate_index):
        # Don't overwrite any data we already have.
        if math.isnan(series.iloc[series_index]):
            series.iloc[series_index] = raw_rates[rate_index]

    safe_add(1 * 12 - 1, '1Y')
    safe_add(2 * 12 - 1, '2Y')
    safe_add(3 * 12 - 1, '3Y')
    safe_add(4 * 12 - 1, '4Y')
    safe_add(5 * 12 - 1, '5Y')
    safe_add(6 * 12 - 1, '6Y')
    safe_add(7 * 12 - 1, '7Y')
    safe_add(8 * 12 - 1, '8Y')
    safe_add(9 * 12 - 1, '9Y')
    safe_add(10 * 12 - 1, '10Y')
         

def build_yield_curve(raw_rates, yield_curve_size=10*12):
    s = pandas.Series(math.nan, index=numpy.arange(yield_curve_size))

    # We use NaN to indicate "the data needs to be interpolated"
    # We have a few different data series that we splice together.
    splice_data(raw_rates, s)
    
    # This will do linear interpolation where it can.
    s.interpolate(inplace=True)
    
    # But it can still leave us with NaNs at the low end of the range
    s.fillna(method='backfill', inplace=True)
    
    return s.tolist()

In [7]:
['%.3f' % (s*100) for s in build_yield_curve(all_by_month.iloc[0])]

['10.327',
 '10.327',
 '10.327',
 '10.327',
 '10.327',
 '10.327',
 '10.327',
 '10.327',
 '10.327',
 '10.327',
 '10.327',
 '10.327',
 '10.247',
 '10.166',
 '10.086',
 '10.005',
 '9.925',
 '9.845',
 '9.764',
 '9.684',
 '9.603',
 '9.523',
 '9.442',
 '9.362',
 '9.318',
 '9.273',
 '9.229',
 '9.185',
 '9.140',
 '9.096',
 '9.052',
 '9.007',
 '8.963',
 '8.919',
 '8.874',
 '8.830',
 '8.804',
 '8.777',
 '8.751',
 '8.725',
 '8.699',
 '8.672',
 '8.646',
 '8.620',
 '8.594',
 '8.568',
 '8.541',
 '8.515',
 '8.501',
 '8.487',
 '8.473',
 '8.459',
 '8.445',
 '8.431',
 '8.418',
 '8.404',
 '8.390',
 '8.376',
 '8.362',
 '8.348',
 '8.343',
 '8.338',
 '8.333',
 '8.329',
 '8.324',
 '8.319',
 '8.314',
 '8.309',
 '8.304',
 '8.300',
 '8.295',
 '8.290',
 '8.286',
 '8.282',
 '8.277',
 '8.273',
 '8.269',
 '8.265',
 '8.261',
 '8.257',
 '8.252',
 '8.248',
 '8.244',
 '8.240',
 '8.230',
 '8.220',
 '8.210',
 '8.200',
 '8.190',
 '8.181',
 '8.171',
 '8.161',
 '8.151',
 '8.141',
 '8.131',
 '8.121',
 '8.122',
 '8.122',
 '8.

In [8]:
bootstrap(build_yield_curve(all_by_month.iloc[-2]), 10 * 12, 4 * 12)

120-48 Ladder { Num Bonds: 73. }

In [9]:
def loop(ladder, rates, max_maturity):
    df = pandas.DataFrame(columns=['NAV', 'Change'])

    # The first iterations have fake data with duplicate years
    # But that's okay because we overwrite them with later data
    # (since they all have the same year)
    for (year, current_rates) in rates:
        if year.year % 5 == 0 and year.month == 1:
            print('Calculating...', year.year)
        (ladder, nav) = iterate_fund(ladder, build_yield_curve(current_rates), max_maturity)
        df.loc[year] = {'NAV' : nav, 'Change' : None}

    calculate_returns(df)
    return df

def calculate_returns(df):
    # Longinvest calculates the return based on comparison's to
    # next year's NAV. So I'll do the same. Even though that seems
    # weird to me. Maybe it's because the rates are based on January?
    # Hmmm...that sounds plausible.
    max_row = df.shape[0]

    for i in range(max_row - 1):
        next_nav = df.iloc[i+1]['NAV']
        nav = df.iloc[i]['NAV']
        change = (next_nav - nav) / nav
        df.iloc[i]['Change'] = change
    return df

def make_annual_ladder(max_maturity, min_maturity, yields):
    rate = yields[max_maturity - 1]
    
    # We have to add the "- 12" in order to make things like up with how
    # longinvest runs things. His "10-4" ladder is really more of "10-3" ladder:
    # bonds get sold the moment they become a 3 year bond.
    ladder = BondLadder(min_maturity - 12, max_maturity)

    face_value = 50
    for i in range(min_maturity, max_maturity + 1, 12):
        ladder.add_bond(Bond(face_value, rate, i))
        face_value = face_value * (1 + rate)

    return ladder

def simulate_monthly_turnover(max_maturity, min_maturity, rates):
    min_maturity = min_maturity * 12
    max_maturity = max_maturity * 12

    initial_yields = build_yield_curve(rates.iloc[0])
    ladder = bootstrap(initial_yields, max_maturity, min_maturity)

    return loop(ladder, rates.iterrows(), max_maturity)

def simulate_annual_turnover(max_maturity, min_maturity, rates):
    min_maturity = min_maturity * 12
    max_maturity = max_maturity * 12

    initial_yields = build_yield_curve(rates.iloc[0])
    ladder = make_annual_ladder(max_maturity, min_maturity, initial_yields)

    # longinvest actually simulates 1870 and assumes 1871 rates. That's why,
    # when the simulation starts in January 1871, all the bonds have paid 1 year
    # of interest and one of the bonds is ready to be sold.
    # So we need to generate 11 months of fake data to do the same simulation.
    # Why 11 months? The 12th month is the real January 1871 data.
    first_index = rates.head(1).index
    bootstrap_rates = itertools.repeat(next(rates.iterrows()), 11)
    all_rates = itertools.chain(bootstrap_rates, rates.iterrows())

    return loop(ladder, all_rates, max_maturity)

In [10]:
%%time
#sim_results = simulate_annual_turnover(10, 4, all_by_month)
sim_results = simulate_monthly_turnover(5, 1, all_by_month)
print(sim_results.head())

/Users/justus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: numpy.pv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pv instead (https://pypi.org/project/numpy-financial/).


Calculating... 1975
Calculating... 1980
Calculating... 1985
Calculating... 1990
Calculating... 1995
Calculating... 2000
Calculating... 2005
Calculating... 2010
Calculating... 2015


/Users/justus/anaconda3/lib/python3.7/site-packages/numpy/lib/financial.py:627: RuntimeWarning: invalid value encountered in double_scalars
  fact = np.where(rate == 0, nper, (1+rate*when)*(temp-1)/rate)


Calculating... 2020
                    NAV    Change
1974-09-01  2875.909128  0.006411
1974-10-01  2894.346908  0.005694
1974-11-01  2910.828462  0.005498
1974-12-01  2926.833529  0.005792
1975-01-01  2943.787114  0.005851
CPU times: user 5.74 s, sys: 96.5 ms, total: 5.83 s
Wall time: 6.9 s


In [11]:
sim_results.to_csv('bonds-monthly-japan.csv')

In [12]:
def get_morningstar(secid):
    url = 'http://mschart.morningstar.com/chartweb/defaultChart?type=getcc&secids=%s&dataid=8225&startdate=1900-01-01&enddate=2016-11-18&currency=&format=1' % secid
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)

    # TODO: why doesn't this work!?!
    r = session.get(url)
    #r = requests.get(url)
    j = r.json()
    
    # The Morningstar data is pretty deeply nested....
    m = j['data']['r'][0]
    assert m['i'] == secid
    
    actual_data = m['t'][0]['d']
    # convert from strings to real data types
    as_dict = dict([(datetime.datetime.strptime(n['i'], '%Y-%m-%d'), float(n['v'])) for n in m['t'][0]['d']])
    
    # Strip out data?
    # Do we only want start of month, end of month, start of year, end of year, etc?
    s = pandas.Series(as_dict, name=secid)

    return s

barclays_index = get_morningstar('XIUSA000MJ')

# Use only final value for each calendar year
def annual(series):
    return series.groupby(by=lambda x: x.year).last()
# Use only final value for each calendar month
def monthly(series):
    return series.groupby(by=lambda x: datetime.date(x.year, x.month, 1)).last()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
monthly_results = pandas.concat([sim_results, monthly(barclays_index)], axis=1, join='outer')

def calculate_change_prev(df, column):
    max_row = df.shape[0]
    
    series = pandas.Series()

    for i in range(max_row - 1):
        val = df.iloc[i][column]
        prev_val = df.iloc[i-1][column]
        change = (val - prev_val) / prev_val
        series.loc[df.iloc[i].name] = change
    return series

def calculate_change_next(df, column):
    max_row = df.shape[0]
    
    series = pandas.Series()

    for i in range(max_row - 1):
        val = df.iloc[i][column]
        next_val = df.iloc[i+1][column]
        change = (next_val - val) / val
        series.loc[df.iloc[i].name] = change
    return series

index_change = calculate_change_prev(monthly_results, 'XIUSA000MJ')
monthly_results = monthly_results.assign(index_change=index_change)
print(monthly_results[["Change", "index_change"]].corr())

In [ ]:
# This is correct when doing monthly calculations.
annual_results = monthly_results.groupby(by=lambda x: x.year).first()
s_nav = calculate_change_next(annual_results, 'NAV')
s_ind = calculate_change_next(annual_results, 'XIUSA000MJ')
annual_results = annual_results.assign(Change=s_nav, index_change=s_ind)
print(annual_results[["Change", "index_change"]].corr())
annual_results.to_csv('bonds-monthly-annual.csv')
print(annual_results.head())